In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer,AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

Load The Dataset

In [ ]:
df = pd.read_csv("CozmoX Assignment Dataset2.csv")
df.head()

,uk_texts,us_texts
0,I colour the centre of my favourite book.,I color the center of my favorite book.
1,He is travelling to the theater.,He is traveling to the theater.
2,I have a flat near the lift.,I have an apartment near the elevator.
3,The programme will start at 6 O'clock.,The program will start at 6 o'clock.
4,He has a cheque for payment.,He has a check for payment.


Data Cleaning

In [ ]:
def clean_text(text):
    text = text.strip()
    return text

In [ ]:
uk_texts = df["uk_texts"].tolist()
us_texts = df["us_texts"].tolist()

In [ ]:
train_uk, test_uk, train_us, test_us = train_test_split(uk_texts, us_texts, test_size=0.2, random_state=42)

Tokenaize the data

In [ ]:
#Load the T5 tokanizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#Tokanize the dataset
class UKUSDataset(object):
    def __init__(self, uk_texts, us_texts, tokenizer, max_length=128):
        self.uk_texts = uk_texts
        self.us_texts = us_texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.uk_texts)
    def __getitem__(self, idx):
        uk_text = self.uk_texts[idx]
        us_text = self.us_texts[idx]
        inputs = self.tokenizer(uk_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        targets = self.tokenizer(us_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze()
        }

In [ ]:
#create datasets
train_df = UKUSDataset(train_uk, train_us, tokenizer)
test_df = UKUSDataset(test_uk, test_us, tokenizer)

In [ ]:
train_loader = DataLoader(train_df, batch_size=8, shuffle=True)
test_loader = DataLoader(test_df, batch_size=8, shuffle=False)

Load The pre-trained transformer (T5) model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training Loop

In [ ]:
def train(model, dataloader, optimizer, device, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

             # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

# Train the model
train(model, train_loader, optimizer, device, epochs=5)


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1, Loss: 10.208561038970947
Epoch 2, Loss: 5.710312032699585
Epoch 3, Loss: 2.6243826746940613
Epoch 4, Loss: 1.4766304612159729
Epoch 5, Loss: 1.3053176522254943


In [ ]:
!pip install evaluate
from evaluate import load

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.0 MB/s eta 0:00:00


In [ ]:
# Load the BLEU metric
bleu_metric = load("bleu")

def evaluate(model, dataloader, device):
    model.eval()
    predictions = []
    references = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Generate predictions
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
            preds = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
            refs = [tokenizer.decode(l, skip_special_tokens=True) for l in labels]
            # Filter out empty references
            filtered_preds = []
            filtered_refs = []
            for pred, ref in zip(preds, refs):
                if len(ref) > 0:  # Only include non-empty references
                    filtered_preds.append(pred)
                    filtered_refs.append([ref])  # BLEU expects list of references

            predictions.extend(preds)
            references.extend([[r] for r in refs])  # BLEU expects list of references

        # Compute BLEU score only if there are valid predictions and references
        if predictions and references:
            bleu_score = bleu_metric.compute(predictions=predictions, references=references)
            print(f"BLEU Score: {bleu_score['bleu']}")
        else:
            print("No valid predictions and/or references for BLEU computation.")

In [ ]:
# Save the model
model.save_pretrained("uk_us_translator")
tokenizer.save_pretrained("uk_us_translator")

('uk_us_translator/tokenizer_config.json',
 'uk_us_translator/special_tokens_map.json',
 'uk_us_translator/spiece.model',
 'uk_us_translator/added_tokens.json')

In [ ]:
def translate(text, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(device)
    generated_ids = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=128)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)


In [ ]:
translated_text = translate("I have a flat near the lift", model, tokenizer, device)
print(translated_text)

In [ ]:
# Test tokenizer
sample_uk_text = "I travelled to the centre of the city."
inputs = tokenizer(
    sample_uk_text,
    return_tensors="pt",
    max_length=128,
    truncation=True,
    padding="max_length"
)

print("Input IDs:", inputs["input_ids"])
print("Attention Mask:", inputs["attention_mask"])

# Test model forward pass
outputs = model(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device), labels=inputs["input_ids"].to(device))
print("Loss:", outputs.loss.item())

# Test model generation
model.eval()
generated_ids = model.generate(inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device), max_length=128)
output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Input:", sample_uk_text)
print("Output:", output_text)

Input IDs: tensor([[   27,     3, 21043,    12,     8,  2050,    13,     8,   690,     5,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     

In [ ]:
!git config --global user.name "ritikashinde"
!git config --global user.email "ritikashinde776@gmail.com"


In [ ]:
!git clone https://github.com/ritikashinde/uk-to-us-converter.git


Cloning into 'uk-to-us-converter'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [ ]:
!ls


'CozmoX Assignment Dataset2.csv'   sample_data	 uk-to-us-converter   uk_us_translator


In [ ]:
!mv uk-to-us-converter.ipynb uk-to-us-converter/


mv: cannot stat 'uk-to-us-converter.ipynb': No such file or directory


In [ ]:
!ls /content


'CozmoX Assignment Dataset2.csv'   sample_data	 uk-to-us-converter   uk_us_translator


In [ ]:
!mv /content/uk_to_us_converter.ipynb /content/uk-to-us-converter/



mv: cannot stat '/content/uk_to_us_converter.ipynb': No such file or directory
